<a href="https://colab.research.google.com/github/Taka0007/Code-Place/blob/main/%E5%8D%98%E4%BD%93%E6%B3%95colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
# @title Simplex method

import numpy as np
import random

class Simplex:
  def __init__(self, condition, objective, ranks_of_A, ranks_of_b):
    
    self.condition, self.objective, self.ranks_of_A, self.ranks_of_b = condition, objective, ranks_of_A, ranks_of_b  
    #↑条件関数＆目的関数などの代入
    self.row_cond      = condition.shape[0]  #条件関数の行数  
    self.list_cond     = condition.shape[1]  #条件関数の列数
    self.basic_var_ind = []                 #基底変数のインデックス
    self.basic_var     = np.empty((self.row_cond,self.list_cond))#基底変数行列
    self.non_basic_ind = []                #非基底変数のインデックス（今はまだ空）
    self.non_basic     = np.empty((self.row_cond, self.list_cond))#非基底変数行列
    
    #以下の変数は後程更新される暫定的な変数
    self.newranks          = np.empty((self.row_cond, self.list_cond))
    self.next_basic_vars   = 0
    self.newA              = np.empty((self.row_cond, self.list_cond))
    self.newb              = np.empty((self.row_cond, self.list_cond))

#線形独立なベクトルを選んでくる関数
  def determin_basic_var(self):
    self.basic_var         = self.condition[:,0:1]  #基底変数に条件関数の1列目を付け加えている(後ろのスライスをミスると、1次元になってしまいエラーをはくので注意)
    new_judgement_ranks    = self.basic_var


    for i in range(1,self.list_cond):
      new_judgement_ranks  = np.hstack((self.basic_var,self.condition[:,i:i+1]))   #列を追加して判定用行列に付け加えている


      #判定用の行列のrankに新たな列を付け加えたもののrankが、その行列全体の列数と同じになる限り付け加え続ける
      #このループが終わった後には、線型独立のものしか残らない（線型従属なものを加えてしまった場合、rank=列数になることはありえないため）
      if np.linalg.matrix_rank(new_judgement_ranks)== new_judgement_ranks.shape[1]:      
        self.basic_var_ind.append(i)
        #judgement_ranks  = np.hstack((judgement_ranks,self.condition[i]))
        self.basic_var   = np.hstack((self.basic_var,self.condition[:,i:i+1]))

      else:
        self.non_basic_ind.append(i)
      
        if len(self.non_basic_ind)==0:
          
          self.non_basic = self.condition[:,i:i+1]

        else:
          self.non_basic = np.hstack((self.non_basic,self.condition[:,i:i+1]))


    self.newA            = (np.linalg.inv(self.basic_var)) @ self.ranks_of_A
    self.newb            = (np.linalg.inv(self.basic_var)) @ self.ranks_of_b

    return self.basic_var


  def optimizeable(self):   #最適化の余地があるかを判定。すでに最適化されていたらFalseを返す
    if np.amin(self.objective) > 0:
      return False

  def next_basic_vars(self):    #目的関数の中で最小の列の添字を返す。これが次の基底変数となる。
    return np.argmin(self.objective)


  def next_nonbasic_vars(self):  #newb/newAが最小の列の添え字を返す。これとnext_basic_varsを入れ替えてあげる。
    
    K                     = Simplex.next_basic_vars(self)
    self.newranks         = self.ranks_of_b / self.ranks_of_A[:, K:K+1]
    self.next_nonbasic_vars  = np.argmin(self.newranks)
    return self.next_nonbasic_vars


  def renew_basic_vars(self):
    Simplex.next_basic_vars(self).add(self.basic_vars_ind)
    Simplex.next_nonbasic_vars(self).remove(self.basic_vars_ind)

  #def reduce_row():    #掃き出し




if __name__ == "__main__":
    
  condition  = np.array([[5,2,1,0,30],[1,2,0,1,14]]) #条件関数
  objective  = np.array([-5,-4,0,0,0])  #目的関数
  ranks_of_A = np.array([[5,2,1,0],[1,2,0,1]])#条件関数の右辺
  ranks_of_b = np.array([[30],[14]])          #条件関数の左辺だけ抽出したもの

  simplex = Simplex(condition, objective, ranks_of_A, ranks_of_b)
  
#  while(simplex.optimizeable()):
  
  
  #print(simplex.determin_basic_var())
  print(simplex.next_nonbasic_vars())

(0, array([[ 6.],
       [14.]]))
